In [92]:
import pandas as pd
import numpy as np
import torch
import embed
import math

In [107]:
dfpdb = pd.read_parquet("./datasets/pdb-no-model-no-asm.parquet")

In [111]:
dfpdb = dfpdb.drop_duplicates("AA")

In [112]:
def num_embeddings(df):
    return df["3Di"].apply(lambda x: math.ceil(len(x)/128)).sum()

In [113]:
num_embeddings(dfpdb)

382511

In [114]:
embed_len = dfpdb["3Di"].apply(lambda x: math.ceil(len(x)/128))
_len = dfpdb["3Di"].apply(lambda x: len(x))


In [126]:
sub = dfpdb[(_len >= 100) & (_len <= 128*4)].copy()
num_embeddings(sub)
sub["len"] = sub["3Di"].apply(lambda x: len(x))

In [118]:
device="cuda"
model = embed.load_model("./checkpoint-large-3.pt", device)
model = model.eval().to(device)

In [133]:
emebds, names = embed.to_embeddings_fast(sub, model, 256, device)

100%|██████████| 3503/3503 [05:11<00:00, 11.26it/s]


In [135]:
pd.DataFrame({"embed": emebds, "name": names}).to_parquet("embed-sub.parquet", index=False)

In [136]:
dfv = pd.read_parquet("./datasets/all-venome.parquet")

In [137]:
emebds, names = embed.to_embeddings_fast(dfv, model, 256, device)

100%|██████████| 282/282 [00:07<00:00, 38.17it/s]


In [138]:
pd.DataFrame({"embed": emebds, "name": names}).to_parquet("embed-venome.parquet", index=False)